In [2]:
import pandas as pd
import numpy as np
import datetime as dt


In [3]:
usersDataTrain = pd.read_csv("data/Eventos(+completo).csv")
usersDataTrain.columns


/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (3,5,6,7,8,9,10,11,12,13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Index(['timestamp', 'event', 'person', 'url', 'sku', 'model', 'condition',
       'storage', 'color', 'skus', 'search_term', 'staticpage',
       'campaign_source', 'search_engine', 'channel', 'new_vs_returning',
       'city', 'region', 'country', 'device_type', 'screen_resolution',
       'operating_system_version', 'browser_version', 'Parte del dia'],
      dtype='object')

In [4]:
def datetime(date,format_string):
    return pd.to_datetime(date,format= format_string,errors='coerce')

usersDataTrain["timestamp"]= datetime(usersDataTrain["timestamp"],"%Y-%m-%dT%H:%M:%S")


In [5]:
usersDataTrain["month"] = usersDataTrain["timestamp"].map(lambda x: x.month)
usersDataTrain["day"] = usersDataTrain["timestamp"].map(lambda x: x.day)
usersDataTrain = usersDataTrain[usersDataTrain["month"] == 5]
usersDataTrain = usersDataTrain[usersDataTrain["day"] >21]

In [6]:
eventosData = usersDataTrain
eventosData = eventosData[['person','event']]
eventosData = pd.get_dummies(eventosData, columns=['event'])
eventosData = eventosData.groupby('person').agg({'event_ad campaign hit':'sum', 'event_brand listing':'sum',
       'event_checkout':'sum', 'event_conversion':'sum', 'event_generic listing':'sum',
       'event_lead':'sum', 'event_search engine hit':'sum', 'event_searched products':'sum',
       'event_staticpage':'sum', 'event_viewed product':'sum', 'event_visited site':'sum'})

In [7]:
eventosData.replace(0,np.nan,inplace = True)
eventosData[eventosData.columns].mean()

event_ad campaign hit       3.633496
event_brand listing         6.800610
event_checkout              1.552333
event_conversion            1.387586
event_generic listing       3.726851
event_lead                  1.409091
event_search engine hit     2.767946
event_searched products     6.930053
event_staticpage            2.128850
event_viewed product       19.089104
event_visited site          2.832423
dtype: float64

In [9]:
#Vamos a mapear si son mayores al promedio de la columna
eventosData['usuario_Checkeador'] = eventosData['event_checkout'].map(lambda x: 1 if (x >=   1.552333) else 0)
eventosData['usuario_susceptible_campaign'] = eventosData['event_ad campaign hit'].map(lambda x: 1 if (x >=  3.633496) else 0)
eventosData['usuario_convertidor'] = eventosData['event_conversion'].map(lambda x: 1 if (x >=  1.387586) else 0)
eventosData['usuario_buscador'] = eventosData['event_viewed product'].map(lambda x:1 if (x>= 19.089104) else 0)
eventosData['usuario_googleador'] = eventosData['event_search engine hit'].map(lambda x: 1 if(x>=2.767946 ) else 0)
eventosData['usuario_dubitativo']= ((eventosData['usuario_Checkeador'] + eventosData['usuario_buscador'] )== 2)
eventosData['usuario_poco_util'] =((eventosData['usuario_Checkeador'] + eventosData['usuario_buscador'] )== 0)
eventosData['usuario_recurrente'] =  eventosData['event_visited site'].map(lambda x: 1 if (x >=  2.832423) else 0)
eventosData['usuario_muy_activo']=((eventosData['usuario_Checkeador'] + eventosData['usuario_buscador']+eventosData['usuario_convertidor'] )== 3)
eventosData = eventosData[['usuario_Checkeador','usuario_susceptible_campaign','usuario_buscador','usuario_dubitativo','usuario_poco_util','usuario_muy_activo','usuario_convertidor']]
eventosData = eventosData.fillna(0)

In [11]:
usersDataTrain = pd.read_csv("setEntrenamiento.csv")
usersDataTrain = pd.merge(usersDataTrain,eventosData.reset_index(),how = "left",on = "person")
usersDataTrain=usersDataTrain.fillna(0)
usersDataTrain.to_csv("setEntrenamiento.csv",index = False)

In [12]:
test = pd.read_csv('setTest.csv')
test = pd.merge(test,eventosData.reset_index(),how = "left",on = "person")
test =test.fillna(0)
test.to_csv("setTest.csv",index = False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19414 entries, 0 to 19413
Data columns (total 96 columns):
person                               19414 non-null object
cant_seciones                        19414 non-null int64
event_checkout                       19414 non-null float64
event_conversion                     19414 non-null float64
event_ad campaign hit                19414 non-null float64
event_lead                           19414 non-null float64
skus                                 19414 non-null int64
screen_resolution                    19414 non-null float64
Busca_terminos_populares             19414 non-null int64
Computer                             19414 non-null float64
Smartphone                           19414 non-null float64
otros                                19414 non-null float64
marca_apple                          19414 non-null float64
marca_asus                           19414 non-null float64
marca_lenovo                         19414 non-null float6